In [1]:
from libdl import Model, DenseLayer, ReLU
import torch
import numpy as np
from python_scripts.data_functionality import Convolution

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

(60000, 28, 28)


In [2]:
model = Model(2, 1.0, 4, int(45000/4), "", "cross_entropy", 10)

model.add(DenseLayer(784, 400, "dense1", 10))
model.add(ReLU())
model.add(DenseLayer(400, 200, "dense2", 10))
model.add(ReLU())
model.add(DenseLayer(200, 10, "dense3", 10))
model.add(ReLU())


In [3]:
#Download mnist and feed it to the network
from tensorflow.keras.datasets import mnist

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
print(train_data.shape)

train_data = np.reshape(train_data, (train_data.shape[0], -1))

print(train_data.shape)

(60000, 28, 28)
(60000, 784)


In [4]:
from libdl import TensorWrapper
data = TensorWrapper(1, 28, 28, 1, False)
labels = TensorWrapper(1, 1, 1, 1, False)
print(train_data.shape)

(60000, 784)


In [5]:
from libdl import Convolution, ReLU, DenseLayer, CrossEntropy
epochs = 5
lr = 0.0001
batch_size = 2
batch = 0

data.set_tensor(train_data[batch+1:batch + batch_size+1], 28, 28, 1)
labels.set_tensor(train_labels[batch+1:batch + batch_size+1], 1, 1, 1)

#predictions = model.forward(data)
        
conv1 = Convolution("conv1", 3, 16, 0, 1, 1, 16)
relu1 = ReLU()
conv2 = Convolution("conv2", 3, 32, 0, 1, 16, 16)
relu2 = ReLU()
dense1 = DenseLayer(18432, 10000, "dense1", 200)
relu3 = ReLU()
dense2 = DenseLayer(10000, 1000, "dense2", 10)
relu4 = ReLU()
dense3 = DenseLayer(1000, 10, "dense2", 10)

cross_entropy = CrossEntropy(10)

grads = TensorWrapper(1, 1, 1, 1, False)

loss = 0
labels.set_tensor(train_labels[0:0 + batch_size], 1, 1, 1)

print(labels.get_tensor()[0, 0], flush=True)


for epoch in range(3):
    for i in range(0, 60000, batch_size):
        data.set_tensor(train_data[i:i + batch_size+1], 28, 28, 1)
        labels.set_tensor(train_labels[i:i + batch_size], 1, 1, 1)
        
        out = conv1.forward(data)
        out = relu1.forward(out)
        out = conv2.forward(out)
        out = relu2.forward(out)
        out = dense1.forward(out)
        out = relu3.forward(out)
        out = dense2.forward(out)
        out = relu4.forward(out)
        out = dense3.forward(out)
        
        exp = np.exp(out.get_tensor())
        s = np.sum(exp)
        exp /= s
        g = exp
        target = labels.get_tensor()[0, 0]
        loss -= np.log(exp[0, int(target)])
        
        print("Batch: ", i, " Loss: " , loss, flush=True)
        
        
        from_ce = cross_entropy.get_gradient(out.get_tensor(), labels.get_tensor(), 20)

        g[0, int(target)] -= 1
        grads.set_tensor(from_ce, 10, 1, 1)
        
        
        
        grads = dense3.backward(grads, 0.0001)
        grads = relu4.backward(grads, 0.0001)
        grads = dense2.backward(grads, 0.0001)
        grads = relu3.backward(grads, 0.0001)
        grads = dense1.backward(grads, 0.0001)
        grads = relu2.backward(grads, 0.0001)
        grads = conv2.backward(grads, 0.0001)
        grads = relu1.backward(grads, 0.0001)
        grads = conv1.backward(grads, 0.0001)

        if i % 20 == 0:
            print("Epoch: " , i)



label = labels.get_tensor()

arr = out.get_tensor() 
    
arr = np.exp(arr)
s = np.sum(arr)
arr /= s

print("Label: " , label, " Sure: " , arr[0, 0] * 100, "%")
    

5.0
Batch:  0  Loss:  2.3522707156647065
From crossentropy:  [[ 0.11204971  0.08476638  0.10878219  0.10387607  0.10100397 -0.90484715
   0.09698477  0.08160807  0.09996265  0.11581333]] 
From clacs:  [[ 0.11204971  0.08476638  0.10878219  0.10387607  0.10100397 -0.90484715
   0.09698477  0.08160807  0.09996265  0.11581333]]
Epoch:  0
Batch:  2  Loss:  4.673137647256162
From crossentropy:  [[ 0.1126514   0.08344076  0.12127676  0.10394607 -0.90181157  0.10422841
   0.08679232  0.08939863  0.0930245   0.10705273]] 
From clacs:  [[ 0.1126514   0.08344076  0.12127676  0.10394607 -0.90181157  0.10422841
   0.08679232  0.08939863  0.0930245   0.10705273]]
Batch:  4  Loss:  6.828587829385775
From crossentropy:  [[ 0.11423353  0.07748997  0.13295496  0.10523309  0.0971521   0.09598211
   0.08322728  0.08572875  0.09214718 -0.88414898]] 
From clacs:  [[ 0.11423353  0.07748997  0.13295496  0.10523309  0.0971521   0.09598211
   0.08322728  0.08572875  0.09214718 -0.88414898]]
Batch:  6  Loss:  9

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in true_divide


Batch:  24  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan nan nan nan]]
Batch:  26  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan nan nan nan]]
Batch:  28  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan nan nan nan]]
Batch:  30  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan nan nan nan]]
Batch:  32  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan nan nan nan]]
Batch:  34  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan nan nan nan]]
Batch:  36  Loss:  nan
From crossentropy:  [[nan nan nan nan nan nan nan nan nan nan]] 
From clacs:  [[nan nan nan nan nan nan nan

KeyboardInterrupt: 

In [ ]:
print(labels.get_tensor())

In [49]:
dense1 = DenseLayer(784, 400, "dense1", 10)
relu1 = ReLU()
dense2 = DenseLayer(400, 200, "dense2", 10)
relu2 = ReLU()
dense3 = DenseLayer(200, 10, "dense3", 10)

for i in range(1000):
    out = dense1.forward(data)
    out = relu1.forward(out)
    out = dense2.forward(out)
    out = relu2.forward(out)
    out = dense3.forward(out)
    
    grads.set_tensor(cross_entropy.get_gradient(out.get_tensor(), labels.get_tensor(), 20), 10, 1, 1)
    
    grads = dense3.backward(grads, 0.0001)
    grads = relu2.backward(grads, 0.0001)
    grads = dense2.backward(grads, 0.0001)
    grads = relu1.backward(grads, 0.0001)
    grads = dense1.backward(grads, 0.0001)
    
    
out = dense1.forward(data)
out = relu1.forward(out)
out = dense2.forward(out)
out = relu2.forward(out)
out = dense3.forward(out)
    
label = labels.get_tensor()

arr = out.get_tensor() 
    
arr = np.exp(arr)
s = np.sum(arr)
arr /= s

print("Label: " , label, " Sure: " , arr[0, 5] * 100, "%")
    

Label:  [[5.]]  Sure:  99.96835927381632 %
